# CSE 601 Data Mining and Bioinformatics

# Project 3: Classification Algorithms

# Part 2: Decision Tree 

Description: This program conducts classification algorithm on datasets (project3_dataset1.txt and project3_dataset2.txt) to find the accuracy on classifying the labels of test dataset.

Submitted By:

Name: Karan Manchandia | UBIT: karanman | Person No.: 50290755

Name: Divya Srivastava | UBIT: divyasri | Person No.: 50290383

Name: Varsha Lakshman | UBIT: varshala | Person No.: 50288138

In [1]:
#Importing libraries
import numpy as np
import sys
import argparse

In [4]:
# Enter the data file name, open it and assign it to the variable data_file
print ("Decision Tree")
while True:
    try:
        filename = input("Enter the name of the data file:")       
        
    # In case of a error the except block will be executed
    except:
        print("Error: Could not find file or read data")
        continue
        
    # This block will be executed after correct data file name is entered
    else:
        print("Correct data file found and imported\n")
        print("----------------------------------------------------------------------------------------\n")
        break

Decision Tree
Enter the name of the data file:project3_dataset4.txt
Correct data file found and imported

----------------------------------------------------------------------------------------



In [5]:
#Opening file handle and retrieving content
with open(filename, "r") as file:
        data_file = [line.rstrip().split('\t') for line in file]


In [6]:
#To check if attribute is String or not
def string_check(s):
    try:
        complex(s)
    except ValueError:
        return True
    return False

In [7]:
#Converting the data of the file to a numpy array 
data_file = np.asarray(data_file)
out_class = data_file[:,-1].reshape((data_file.shape[0],1)).astype(int)
data_file = data_file[:,0:-1]

#Detecting categorical variables in the dataset and giving them numerical values
s_indices = []
indi=[]
i = 0
while i < len(data_file[0]):
    if string_check(data_file[0][i]):
        s_indices.append(i)
    i+=1

for i in s_indices:
    s_unique = np.unique(data_file[:,i])
    print(s_unique)
    substitute_vals = list(range(len(s_unique)))
    dictionary = dict(zip(s_unique, substitute_vals))
    j=0
    while j < len(data_file[:,i]):
        data_file[j][i] = dictionary.get(data_file[j][i])
        j+=1

['overcast' 'rain' 'sunny']
['cool' 'hot' 'mild']
['high' 'normal']
['strong' 'weak']


In [8]:
#Appending the output class at the last 
data_file = data_file.astype(float)
data_file = np.append(data_file,out_class,axis=1)

#Applying 10-fold cross validation
cross_validation = np.array_split(data_file, 10)
accuracy = precision = recall = f1_measure = 0

In [9]:
#Finding the split-node in the decision tree
def find_node(data_file):
    min_errorVal = float('inf')
    root = {}
    for col in range(len(data_file[0]) - 1):
        for row in range(len(data_file)):
            #Function to split the data into two sublists (left_list and right_list)
            left_list = []
            right_list = []
            for r in range(len(data_file)):
                if col not in s_indices:
                    if data_file[r][col] <= data_file[row][col]:
                        left_list.append(data_file[r])
                    else:
                        right_list.append(data_file[r])
                else:
                    if data_file[r][col] == data_file[row][col]:
                        left_list.append(data_file[r])
                    else:
                        right_list.append(data_file[r])
            left_list = np.asarray(left_list)
            right_list = np.asarray(right_list)
            
            #Calculating gini_index
            count = len(left_list) + len(right_list)
            one_count_left = zero_count_left = one_count_right = zero_count_right = 0
            if len(left_list) != 0:
                one_count_left = float(list(left_list[:,-1]).count(1)) /len(left_list)
                zero_count_left = float(list(left_list[:,-1]).count(0)) / len(left_list)
            if len(right_list) != 0:
                one_count_right = float(list(right_list[:,-1]).count(1)) / len(right_list)
                zero_count_right = float(list(right_list[:,-1]).count(0)) / len(right_list)
            gini_left = 1.0 - ((one_count_left * one_count_left) + (zero_count_left * zero_count_left))
            gini_right = 1.0 - ((one_count_right * one_count_right) + (zero_count_right * zero_count_right))
            gini = ((gini_left * len(left_list)) + (gini_right * len(right_list))) / count

            if gini < min_errorVal:
                min_errorVal = gini
                root['split_index'] = row
                root['split_value'] = data_file[row][col]
                root['split_attr'] = col
                root['left_list'] = left_list
                root['right_list'] = right_list
                root['gini'] = min_errorVal
    
#     print("impurity measure is:", min_errorVal)

    return root



In [10]:
#Building the decision tree
def decision_tree(root, depth):
    left_list = root['left_list']
    right_list = root['right_list']
    del(root['left_list'])
    del(root['right_list'])
    if len(left_list) == 0 or len(right_list) == 0:
        root['left_list'] = root['right_list'] = find_class(left_list, right_list)
        return root 
    if len(left_list) > 0:
        #Checking if everything belongs to the same class
        if len(np.unique(left_list[:,-1])) == 1:
            root['left_list'] = find_class(left_list, list())
        else:
            root['left_list'] = decision_tree(find_node(left_list), depth + 1)
    if len(right_list) > 0:
        #Checking if everything belongs to the same class
        if len(np.unique(right_list[:,-1])) == 1:
            root['right_list'] = find_class(list(), right_list)
        else:
            root['right_list'] = decision_tree(find_node(right_list), depth + 1)
    return root


In [11]:
#Printing the tree
def print_tree(root, depth=0):
    if isinstance(root, dict):
        print(depth*'-', 'X', (root['split_index']+1),'<',root['split_value'])
        print_tree(root['left_list'], depth+1)
        print_tree(root['right_list'], depth+1)
    else:
        print(depth*'-', root)


In [12]:
#Get the classification at the node
def find_class(left_list, right_list):
    count_zero = 0
    count_one = 0;
    if len(left_list) != 0:
        count_zero += list(left_list[:,-1]).count(0)
        count_one += list(left_list[:,-1]).count(1)
    if len(right_list) != 0:
        count_zero += list(right_list[:,-1]).count(0)
        count_one += list(right_list[:,-1]).count(1)
    return 1 if count_one > count_zero else 0

In [13]:
#Classifying the test record using the decision tree
def test_classification(root, test_data):
    if root == 0 or root == 1:
        return root
    elif root['split_attr'] not in s_indices:
        if test_data[root['split_attr']] < root['split_value']:
            if root['left_list'] == 0 or root['left_list'] == 1:
                return root['left_list']                                 
            else:
                return test_classification(root['left_list'], test_data)
        else:
            if root['right_list'] == 0 or root['right_list'] == 1:
                return root['right_list']
            else:
                return test_classification(root['right_list'], test_data)
    else:
        if test_data[root['split_attr']] == root['split_value']:
            if root['left_list'] == 0 or root['left_list'] == 1:
                return root['left_list']                                 
            else:
                return test_classification(root['left_list'], test_data)
        else:
            if root['right_list'] == 0 or root['right_list'] == 1:
                return root['right_list']
            else:
                return test_classification(root['right_list'], test_data)

In [14]:
#Calculating the performance parameters and Getting metrics - TruePositive, FalsePositive, TrueNegative, FalseNegative for calculating accuracy/precision/recall/f-measure
def performance_metrics(actual, predicted):
    tp = tn = fp = fn = 0
    for i in range(len(actual)):

        if actual[i] == 1 and predicted[i] == 1:
            tp += 1
        if actual[i] == 1 and predicted[i] == 0:
            fn += 1
        if actual[i] == 0 and predicted[i] == 1:
            fp += 1
        if actual[i] == 0 and predicted[i] == 0:
            tn += 1
    precision = 0
    recall = 0
    F1 = 0
    accuracy = 0

    if tp + fp != 0:
        precision = tp / float(tp + fp)
    if tp + fn != 0:
        recall = tp / float(fn + tp)
    if tp + tn + fp + fn != 0:
        accuracy = (tp + tn) / float(tp + tn + fp + fn)
    if precision + recall != 0:
        F1 = (2 * precision * recall) / float(precision + recall)

    precision = precision * 100
    recall = recall * 100
    accuracy = accuracy * 100
    F1 = F1 * 100
    
    return {'precision': precision, 'recall': recall, 'accuracy': accuracy, 'F1': F1}


In [15]:
#Actual algorithm

scores = []
print(f"Decision Tree for file {filename}")
for i in range(len(cross_validation)):
    #split dataset into train and test - 10 fold cross-validation
    test_set = cross_validation[i]
    train_set = np.vstack([x for l,x in enumerate(cross_validation) if l != i])
    test_set = np.asarray(test_set)
    train_set = np.asarray(train_set)
    test_class = []
    root = find_node(train_set)
    root = decision_tree(root, 1)

    for row in range(len(test_set)):
        test_class.append(test_classification(root,test_set[row]))
    actual_class = test_set[:,-1]
    predicted_class = np.asarray(test_class)
    #Getting metrics - TruePositive, FalsePositive, TrueNegative, FalseNegative for calculating accuracy/precision/recall/f-measure
    accuracy = performance_metrics(actual_class, predicted_class)
    scores.append(accuracy)

print("Tree")
print(root)
print_tree(root)

precision = 0
recall = 0
accuracy = 0
F1 = 0

for i in range(len(scores)):
    precision += scores[i]['precision']
    recall += scores[i]['recall']
    accuracy += scores[i]['accuracy']
    F1 += scores[i]['F1']

print('Final Accuracy: %.3f%%' % (accuracy/float(len(scores))))
print('Final Precision: %.3f%%' % (precision/float(len(scores))))
print('Final Recall: %.3f%%' % (recall/float(len(scores))))
print('Final F1: %.3f%%' % (F1/float(len(scores))))


Decision Tree for file project3_dataset4.txt
Tree
{'split_index': 0, 'split_value': 2.0, 'split_attr': 0, 'gini': 0.31923076923076926, 'left_list': {'split_index': 0, 'split_value': 0.0, 'split_attr': 2, 'gini': 0.0, 'left_list': 0, 'right_list': 1}, 'right_list': {'split_index': 2, 'split_value': 0.0, 'split_attr': 1, 'gini': 0.16666666666666666, 'left_list': {'split_index': 0, 'split_value': 1.0, 'split_attr': 0, 'gini': 0.3333333333333333, 'left_list': {'split_index': 0, 'split_value': 1.0, 'split_attr': 3, 'gini': 0.0, 'left_list': 1, 'right_list': 0}, 'right_list': 1}, 'right_list': 1}}
 X 1 < 2.0
- X 1 < 0.0
-- 0
-- 1
- X 3 < 0.0
-- X 1 < 1.0
--- X 1 < 1.0
---- 1
---- 0
--- 1
-- 1
Final Accuracy: 55.000%
Final Precision: 50.000%
Final Recall: 55.000%
Final F1: 50.000%
